In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

In [ ]:
from tika import parser
from PyPDF2 import PdfFileReader
import filetype
import math
import os
import string
import platform
import operator
import re
from nltk.tokenize import RegexpTokenizer

In [ ]:
class Document:
    def __init__(self, url, length, id, textList):
        self.url = url
        self.length = length
        self.id = id
        self.score = 0.
        self.textList = textList

In [ ]:
class Index:
    hashmap = {} #dictionary
    fileCount = 0 #integer, Gesamtzahl aller gefunden Dateien
    docHashmap = {}

In [ ]:
def buildIndex(self):
    
    #startDirectories = self._getStartDirectories()
    startDirectories = ["F:/Jonas/IT/Springer-Bücher"]
    
    for directory in startDirectories:
        for root, _, files in os.walk(directory):
            for file in files:
                
                path = os.path.abspath(os.path.join(root, file))
                
                try:
                    if filetype.guess(path).mime == 'application/pdf':
                        
                        rawText = ""
                        
                        with open(path, 'rb') as f:
                            pdf = PdfFileReader(f)
                            numberOfPages = pdf.getNumPages()
                            for number in range(numberOfPages):
                                page = pdf.getPage(number)
                                rawText += page.extractText()
                        #fileData = parser.from_file(path)
                        #rawText = fileData['content']
                        self.fileCount += 1
                    
                        processedText = self._preprocessText(rawText)
                        document = Document(path, len(processedText), self.fileCount, processedText)
                        self.docHashmap.update({self.fileCount : document})
                        self._addToIndex(self.fileCount, processedText)
                except:
                    #print(path)
                    continue

Index.buildIndex = buildIndex

In [ ]:
def _getStartDirectories(self):

    if platform.system() == "Linux":
        directories = ["/"]
        
    elif platform.system() == "Darwin":
        directories = ["/"]
        
    elif platform.system() == "Windows":
        directories = ['%s:\\' % d for d in string.ascii_uppercase if os.path.exists('%s:' % d)]
        
    else:
        raise EnvironmentError
        
    return directories

Index._getStartDirectories = _getStartDirectories

In [ ]:
def _addToIndex(self, documentID, terms):
    
    for term in terms:
        
        try:
            docSet = self.hashmap[term]
            docSet.add(documentID)
            self.hashmap.update({term : docSet})
            
        except KeyError:
            docSet = {documentID}
            self.hashmap.update({term : docSet})
    
Index._addToIndex = _addToIndex

In [ ]:
def _preprocessText(self, text):
    
    lowerText = text.lower()
    
    prepText = re.sub(r'\d+', '', lowerText)
            
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+-$|\w+')
    tokenList = tokenizer.tokenize(prepText)
    
    for token in tokenList:
        if token[-1] == '-':
            
            index = tokenList.index(token)
            compositeWord = token[:-1]+tokenList[index+1]
            tokenList[index] = compositeWord
            del tokenList[index+1]
            
    return tokenList
    
Index._preprocessText = _preprocessText

# IMPROVEMENT

In [ ]:
def retrieve(self, searchString):
 
    processedStrings = self._preprocessText(searchString)
    result, intsect = set(), set()
    df, helpDict, helpDict2 = {}, {}, {}
    resultList, resultList2 = [], []
    
    for word in processedStrings:
        try:
            documents = set(self.hashmap[word])
            df[word] = len(documents)
            intsect = result.intersection(documents).union(intsect)
            result = result.union(documents)
            
        except KeyError:
            continue

    for document in result:
        doc = ind.docHashmap[document]
        doc.tf_idf(processedStrings,df)
        helpDict[doc.id] = doc.score
        
    for key in intsect:
        helpDict2[key] = helpDict[key]
        helpDict.pop(key)
        
    sortedDict = sorted(helpDict.items(), key=operator.itemgetter(1))
    sortedDict2 = sorted(helpDict2.items(), key=operator.itemgetter(1))

    
    for key,_ in sortedDict:
        resultList.append(ind.docHashmap[key].url)
    
    for key,_ in sortedDict2:
        resultList2.append(ind.docHashmap[key].url)
    
    
    resultList = resultList[::-1]
    resultList2 = resultList2[::-1]
    
    return resultList2+resultList

Index.retrieve = retrieve

In [ ]:
def tf_idf(self, termList, df):
    
    tfDict = {}
    for term in termList:
        tfDict[term] = 0
    
    ind = Index()
        
    for term in self.textList:
        if term in termList:
            tfDict[term] = tfDict[term]+1

    for key, value in df.items():
        idf = math.log((ind.fileCount/value+1),10)
        tfDict[key] = tfDict[key]*idf
    
    self.score = sum(tfDict.values())

Document.tf_idf = tf_idf

In [ ]:
ind = Index()
ind.buildIndex()

In [ ]:
resultSet = ind.retrieve("sprache")
if resultSet:
    for elem in resultSet:
        if elem.split('.')[-1] != 'dat':
            print(elem)

In [ ]:
for index in [1, 2, 166, 137, 138, 171, 210]:
    print(ind.docHashmap[index].url)

In [ ]:
print(len(ind.docHashmap))